In [ ]:
"""
Created on Wed Nov 22 17:27 2023

Apply weights to future as well

Author: @claraburgard

"""

In [1]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import seaborn as sns
import multimelt.useful_functions as uf

In [2]:
sns.set_context('paper')

In [3]:
%matplotlib qt5

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-burgardc'


READ IN DATA

In [4]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_atmo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'


In [5]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [6]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

In [7]:
param_classic_list = ['linear_local',
              'quadratic_local','quadratic_local_locslope',
              'quadratic_mixed_mean','quadratic_mixed_locslope',
              'lazero19',
              'boxes_4_pismyes_picopno']

param_NN_list = ['NNsmall']

READ IN WEIGHTS

In [8]:
bay_weights = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison.nc')['bay_weights']

In [9]:
bay_weights_meanNisf = bay_weights.mean('Nisf')

In [10]:
plotted_var = bay_weights_meanNisf


x = np.arange(len(plotted_var.param))
y = np.arange(len(plotted_var.model))

fig, axs = plt.subplots(1, 1,figsize=(8.25,8.25))
ax0 = axs.imshow(plotted_var.values, cmap=plt.cm.Greens)
axs.set_yticklabels(labels=plotted_var.model.values)

axs.yaxis.tick_right()
axs.set_yticks(y)
axs.yaxis.tick_right()

axs.xaxis.tick_top()    
axs.set_xticks(x)
axs.set_xticklabels(plotted_var.param.values, rotation=90)


cb_ax = fig.add_axes([0.15, 0.08, 0.7, 0.02])
cbar = fig.colorbar(ax0, cax=cb_ax, extend='both',orientation='horizontal')
plt.tight_layout()
fig.savefig(plot_path+'heatmap_bayesian_weights.pdf')

libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast
/tmp/ipykernel_5081/2539507994.py:9: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs.set_yticklabels(labels=plotted_var.model.values)
/tmp/ipykernel_5081/2539507994.py:22: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [20]:
sens_weights = xr.DataArray(data=np.array([0.11,
                                           0.24,
                                           0.03,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.24,
                                           0.47,
                                           0.41,
                                           0.12,
                                           0.43,
                                           0.39,
                                           0.05]), dims=['model']).assign_coords({'model': 
                                                                                  ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                   'CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                                                                                   'GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                   'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']})

In [21]:
future_weight = bay_weights * sens_weights.sel(model=bay_weights.model)
bay_weights_meanNisf = future_weight.mean('Nisf')

In [22]:
plotted_var = bay_weights_meanNisf


x = np.arange(len(plotted_var.param))
y = np.arange(len(plotted_var.model))

fig, axs = plt.subplots(1, 1,figsize=(8.25,8.25))
ax0 = axs.imshow(plotted_var.values, cmap=plt.cm.Greens)
axs.set_yticklabels(labels=plotted_var.model.values)

axs.yaxis.tick_right()
axs.set_yticks(y)
axs.yaxis.tick_right()

axs.xaxis.tick_top()    
axs.set_xticks(x)
axs.set_xticklabels(plotted_var.param.values, rotation=90)


cb_ax = fig.add_axes([0.15, 0.08, 0.7, 0.02])
cbar = fig.colorbar(ax0, cax=cb_ax, extend='both',orientation='horizontal')
plt.tight_layout()
fig.savefig(plot_path+'heatmap_bayesianandsens_weights.pdf')

/tmp/ipykernel_17883/3314147286.py:9: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs.set_yticklabels(labels=plotted_var.model.values)
/tmp/ipykernel_17883/3314147286.py:22: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
